In [31]:
import numpy as np
from sentence_transformers import SentenceTransformer
import datasets
from datasets import load_from_disk
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader

In [1]:
td = load_from_disk("../train_with_origin_gt_add_top_k_passage/not_include_answer_passage_train_es_top_4")
vd = load_from_disk("../valid_with_origin_gt_add_top_k_passage/not_include_answer_passage_valid_es_top_4")

model = SentenceTransformer("KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS")



In [2]:
import json

with open('../data/preprocess_wiki.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

In [3]:
with open('../hybrid_test_retrieval_v3.bin','rb') as f:
    hb_v3 = pickle.load(f)

with open('../elastic_test_500.bin', 'rb') as f:
    es_500 = pickle.load(f)

test_dataset = load_from_disk('../data/test_dataset')['validation']
test_dataset

Dataset({
    features: ['question', 'id'],
    num_rows: 600
})

In [6]:
with open('../data/sparse_embedding.bin', 'rb') as f:
    sparse_embedding = pickle.load(f)
with open('../data/tfidv.bin', 'rb') as f:
    tfidfv = pickle.load(f)

k = 500

queries = test_dataset['question']
query_vec = tfidfv.transform(queries)
# assert np.sum(query_vec) != 0, "오류가 발생했습니다. 이 오류는 보통 query에 vectorizer의 vocab에 없는 단어만 존재하는 경우 발생합니다."

result = query_vec * sparse_embedding.T
result = result.toarray()

doc_scores = []
doc_indices = []
for i in range(result.shape[0]):
    sorted_result = np.argsort(result[i, :])[::-1]
    doc_scores.append(result[i, :][sorted_result].tolist()[:k])
    doc_indices.append(sorted_result.tolist()[:k])

In [7]:
question_embedding = model.encode(queries)

In [9]:
question_embedding.shape

(600, 768)

In [55]:
t_q = list(es_500.keys())
t_p = list(es_500.values())

In [22]:
len(t_p[0])

500

In [53]:
def cos_sim(q_emb, p_emb):
    return np.dot(q_emb, p_emb) / (np.linalg.norm(q_emb) * np.linalg.norm(p_emb))

In [ ]:
for i in range(30):
    q_emb = model.encode(t_q[0])
    p_emb = model.encode(wiki[str(t_p[0][i])]['text'])
    print(np.dot(q_emb, p_emb) / (np.linalg.norm(q_emb) * np.linalg.norm(p_emb)))

In [58]:
from tqdm import tqdm

ranks = []
for i in tqdm(range(len(t_q))):
    q_emb = model.encode(t_q[i])
    
    rank = []
    for j in range(len(t_p[i])):
        p_emb = model.encode(wiki[str(t_p[i][j])]['text'])
        rank.append(cos_sim(q_emb, p_emb))
    
    ranks.append(rank)

100%|██████████| 600/600 [1:16:01<00:00,  7.60s/it]


In [69]:
t_p[0][0]

43280

## only passage rank

In [62]:
tmp = ranks

In [73]:
t_p[0][0]

43280

In [74]:
passage_rank = {}
for i in range(len(tmp)):
    id_rank_table = {}
    for doc_id, score in zip(t_p[i], tmp[i]):
        id_rank_table[doc_id] = score
    passage_rank[i] = id_rank_table

In [80]:
type(passage_rank[0])

dict

In [ ]:
# a = passage_rank[0]

for i in range(len(passage_rank)):
    passage_rank[i] = dict(sorted(passage_rank[i].items(), key=lambda x : x[1], reverse=True))


In [165]:
import pandas as pd

hb_t = []

for i in range(len(t_q)):
    hb_t.append({
        'id': test_dataset['id'][i],
        'question': t_q[i],
        'context': " ".join([wiki[str(p)]['text'] for p in list(list(passage_rank.values())[i].keys())[:20]])
    })
hb =pd.DataFrame(hb_t)
hb


,id,question,context
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,이 정의는 엄밀해야 할 정의가 모호성을 띄고 있다는 점에서 비판을 많이 받았다. 천...
1,mrc-1-001113,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,"여범규 감독이 새로 부임하고 K리그 챌린지에서 리그를 시작한 광주는 루시오, 김은선..."
2,mrc-0-002191,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,"하원 회의장은 궁전의 북쪽 끝에 위치해 있다. 방의 크기는 14*20.7m이고, 상..."
3,mrc-0-003951,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?","외국어(外國語)는 'foreign language'에 해당하는 말인데, 이 말의 문..."
4,mrc-1-001272,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,이 성의 수도인 뚜옌꽝은 이 지역에서 벌어진 전투의 풍부한 역사를 가지고 있다. 가...
...,...,...,...
595,mrc-0-002989,타입 2 가이아 메모리을 만든 집단은?,"재단 X가 슈라우드(=소노자키 후미네)가 개발한 가이아 메모리의 테크놀로지를 분석,..."
596,mrc-0-001804,장면이 정치보복에 반대하는 입장에서 처벌을 원치 않은 대상은?,이명박 정부의 중간평가 성격이 짙은 것으로 평가되는 지방선거에서 한나라당은 패배를 ...
597,mrc-0-003411,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?",콜드게임(called game)이란 야구나 소프트볼 등의 운동경기에서 심판에 의하여...
598,mrc-0-003436,제2캐나다기갑여단이 상륙한 곳은?,제2캐나다보병사단이 주력 부대로 투입되기 직전이자 영국에 도착하여 편성을 바꾼지 약...


In [169]:
from datasets import Dataset

Dataset.from_pandas(hb).save_to_disk(f"sentence_transformer_k_20")

In [89]:
len(hb['context'][0])

17009

In [90]:
a = load_from_disk('../hb_k_20_v3')
len(a[0]['context'])

24058

In [98]:
(list(es_500.values())[0])[0]

43280

In [110]:
passage_rank_for_save = {t_q[k]: v for k, v in passage_rank.items()}

In [111]:
with open('passage_rank_using_sentence_transformer.pickle', 'wb') as f:
    pickle.dump(passage_rank_for_save, f, pickle.HIGHEST_PROTOCOL)

In [116]:
with open('passage_rank_using_sentence_transformer.pickle', 'rb') as f:
    d = pickle.load(f)

## hybrid filtering for sentence transformers

In [128]:
t_q = list(hb_v3.keys())
t_p = list(hb_v3.values())

In [131]:
from tqdm import tqdm

hb_ranks = []
for i in tqdm(range(len(t_q))):
    q_emb = model.encode(t_q[i])
    
    hb_rank = []
    for j in range(len(t_p[i])):
        p_emb = model.encode(wiki[str(t_p[i][j])]['text'])
        hb_rank.append(cos_sim(q_emb, p_emb))
    
    hb_ranks.append(hb_rank)

100%|██████████| 600/600 [22:31<00:00,  2.25s/it]


In [132]:
tmp_hb = hb_ranks
tmp_hb_rank = {}
for i in range(len(tmp_hb)):
    id_rank_table = {}
    for doc_id, score in zip(t_p[i], tmp_hb[i]):
        id_rank_table[doc_id] = score
    tmp_hb_rank[i] = id_rank_table

for i in range(len(tmp_hb_rank)):
    tmp_hb_rank[i] = dict(sorted(tmp_hb_rank[i].items(), key=lambda x : x[1], reverse=True))

In [164]:
hhh_t = []
for i in range(len(t_q)):
    hhh_t.append({
        'id': test_dataset['id'][i],
        'question': t_q[i],
        'context': " ".join([wiki[str(p)]['text'] for p in list(list(tmp_hb_rank.values())[i].keys())[:20]])
    })
hhh_t =pd.DataFrame(hhh_t)
hhh_t

,id,question,context
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,이 정의는 엄밀해야 할 정의가 모호성을 띄고 있다는 점에서 비판을 많이 받았다. 천...
1,mrc-1-001113,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,"여범규 감독이 새로 부임하고 K리그 챌린지에서 리그를 시작한 광주는 루시오, 김은선..."
2,mrc-0-002191,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,"하원 회의장은 궁전의 북쪽 끝에 위치해 있다. 방의 크기는 14*20.7m이고, 상..."
3,mrc-0-003951,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?","외국어(外國語)는 'foreign language'에 해당하는 말인데, 이 말의 문..."
4,mrc-1-001272,디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?,"과거 홍강 삼각주의 중심부에 자리 잡고 있었지만, 과거에는 타이빈성은 3개의 큰 강..."
...,...,...,...
595,mrc-0-002989,타입 2 가이아 메모리을 만든 집단은?,"재단 X가 슈라우드(=소노자키 후미네)가 개발한 가이아 메모리의 테크놀로지를 분석,..."
596,mrc-0-001804,장면이 정치보복에 반대하는 입장에서 처벌을 원치 않은 대상은?,이명박 정부의 중간평가 성격이 짙은 것으로 평가되는 지방선거에서 한나라당은 패배를 ...
597,mrc-0-003411,"콜드게임 중 어떠한 계기로 인해 잠시 중단된 뒤, 익일에 게임이 진행되는 것은?",콜드게임(called game)이란 야구나 소프트볼 등의 운동경기에서 심판에 의하여...
598,mrc-0-003436,제2캐나다기갑여단이 상륙한 곳은?,제2캐나다보병사단이 주력 부대로 투입되기 직전이자 영국에 도착하여 편성을 바꾼지 약...


In [175]:
hb_rank_for_save = {t_q[k]: v for k, v in tmp_hb_rank.items()}

with open('hybrid_filtering_using_sentence_transformer.pickle', 'wb') as f:
    pickle.dump(hb_rank_for_save, f, pickle.HIGHEST_PROTOCOL)

In [177]:
with open('hybrid_filtering_using_sentence_transformer.pickle', 'rb') as f:
    dd = pickle.load(f)
len(list(dd.values())[0])

100

In [168]:
len(hb['context'][0]), len(hhh_t['context'][0])

(16290, 18156)

In [172]:
Dataset.from_pandas(hhh_t).save_to_disk(f"sentence_transformer_k_20_hybrid_v3")

In [174]:
yyy = load_from_disk('../hb_k_20_v3')
len(yyy[0]['context'])

24058